# Day 2: Data Cleaning & Exploration

## Objective
Ensure the dataset is fully prepared for modeling. This includes checking and fixing structure, column names, data types, distributions, potential label leakage, and fairness risks.

## Key Steps
- Load and inspect data
- Standardize column names and formats
- Assess and correct data types
- Confirm completeness and consistency
- Review class balance
- Flag or drop low-variance and irrelevant features
- Add notes on human-centered relevance
- Save cleaned dataset

## Loading and inspecting the dataset

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("../data/gallstone.xlsx")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### Initial Data Snapshot

The dataset contains 319 rows and 38 columns. It includes demographic, lab, and bioimpedance measurements. No missing values were found. The target variable (`gallstone_status`) is well balanced: 161 cases, 158 controls.

## Standardizing Column Names

We standardized column names to lowercase snake_case for readability and consistency. For example:
- `Total Body Water (TBW)` → `total_body_water_tbw`
- `C-reactive protein (CRP)` → `crp`

In [ ]:
import re

def clean_column_name(col):
    col = col.lower()
    col = re.sub(r"[^\w\s]", "", col)          # Remove special characters like ()%,$
    col = re.sub(r"\s+", "_", col)             # Replace spaces/tabs with underscores
    col = re.sub(r"_+", "_", col)              # Collapse multiple underscores
    col = col.strip("_")                       # Remove leading/trailing underscores
    return col

df.columns = [clean_column_name(col) for col in df.columns]

In [ ]:
rename_map = {
    "coronary_artery_disease_cad": "cad",
    "diabetes_mellitus_dm": "diabetes",
    "height": "height_cm",
    "weight": "weight_kg",
    "body_mass_index_bmi": "bmi",
    "total_body_water_tbw": "tbw_kg",
    "extracellular_water_ecw": "ecw_kg",
    "intracellular_water_icw": "icw_kg",
    "extracellular_fluidtotal_body_water_ecftbw": "ecf_tbw_ratio",
    "total_body_fat_ratio_tbfr": "fat_ratio_percent",
    "lean_mass_lm": "lean_mass_percent",
    "body_protein_content_protein": "protein_percent",
    "visceral_fat_rating_vfr": "vfr",
    "bone_mass_bm": "bone_mass_kg",
    "muscle_mass_mm": "muscle_mass_kg",
    "obesity": "obesity_percent",
    "total_fat_content_tfc": "fat_mass_kg",
    "visceral_fat_area_vfa": "visceral_fat_area_cm2",
    "visceral_muscle_area_vma_kg": "visceral_muscle_mass_kg",
    "hepatic_fat_accumulation_hfa": "hepatic_fat",
    "glucose": "glucose_mg_dl",
    "total_cholesterol_tc": "cholesterol_total_mg_dl",
    "low_density_lipoprotein_ldl": "ldl_mg_dl",
    "high_density_lipoprotein_hdl": "hdl_mg_dl",
    "triglyceride": "triglyceride_mg_dl",
    "aspartat_aminotransferaz_ast": "ast_u_l",
    "alanin_aminotransferaz_alt": "alt_u_l",
    "alkaline_phosphatase_alp": "alp_u_l",
    "creatinine": "creatinine_mg_dl",
    "glomerular_filtration_rate_gfr": "gfr_ml_min",
    "creactive_protein_crp": "crp_mg_l",
    "hemoglobin_hgb": "hemoglobin_g_dl",
    "vitamin_d": "vitamin_d_ng_ml"
}

df.rename(columns=rename_map, inplace=True)

In [ ]:
df.rename(columns={"gallstone_status": "has_gallstones"}, inplace=True)

In [ ]:
df.rename(columns={
    "vfr": "vfr_score",
    "ecf_tbw_ratio": "ecf_tbw_ratio_index"
}, inplace=True)

In [ ]:
df.columns.to_list()

## Handling Missing Values

We found missing values.

In [ ]:
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_summary[missing_summary > 0]
df.isnull().sum().sum()

## Data type correction

Several binary clinical indicators (e.g., `diabetes_mellitus_dm`, `hypothyroidism`) were stored as numeric. We cast them to categorical for clarity and to prevent incorrect modeling treatment as continuous features.

The following columns were treated as categorical:

- `gender`
- `comorbidity`
- `cad`
- `hypothyroidism`
- `hyperlipidemia`
- `diabetes`
- `hepatic_fat`
- `has_gallstones`

These are discrete features important to clinical understanding.

In [ ]:
# Example for known binary or categorical fields
categorical_cols = [
    "gender",
    "comorbidity",
    "cad",
    "hypothyroidism",
    "hyperlipidemia",
    "diabetes",
    "hepatic_fat",
    "has_gallstones"
]

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype("category")

In [ ]:
df.dtypes.value_counts()

## Class balance check

We need to verify whether the dataset has a balanced distribution of the target variable (`has_gallstones`). A balanced target avoids biasing the model toward the majority class. Since this is a binary classification task, a roughly 50/50 split is ideal.

In [ ]:
# Raw counts
df["has_gallstones"].value_counts()

In [ ]:
# Proportions
df["has_gallstones"].value_counts(normalize=True)

In [ ]:
# Rename for readability
df["has_gallstones"] = df["has_gallstones"].cat.rename_categories({0: "No Gallstones", 1: "Gallstones"})

In [ ]:
# Bar plot (1 output per cell – show plot only)
import matplotlib.pyplot as plt

df["has_gallstones"].value_counts().plot(kind="bar", color=["skyblue", "salmon"])
plt.title("Class Balance: Gallstone Presence")
plt.ylabel("Number of Patients")
plt.xlabel("Gallstone Status")
plt.grid(True)
plt.tight_layout()
plt.show()

The dataset includes 161 patients with gallstones and 158 without. The distribution is:
- Gallstones: 50.5%
- No Gallstones: 49.5%

This class balance is excellent. It allows us to:
- Use standard accuracy and ROC-AUC as reliable evaluation metrics
- Avoid applying class weighting or resampling
- Expect fair learning behavior from most classification models

## Dropping Constant or Near-Constant Columns

We remove columns that contain the same value in nearly all rows. These features add no predictive power and can distort feature selection, particularly for tree-based models.

We define near-constant as having the same value in over 99% of rows.

In [ ]:
# Threshold: 99% of the same value across all rows
threshold = 0.99
constant_like_cols = []

for col in df.columns:
    top_freq = df[col].value_counts(normalize=True).max()
    if top_freq >= threshold:
        constant_like_cols.append(col)

constant_like_cols

In [ ]:
df.drop(columns=constant_like_cols, inplace=True)

No columns were dropped because they were constant or nearly constant in over 99% of the dataset:

Removing them:
- Improves model performance by eliminating noise
- Speeds up training
- Prevents misleading feature importance metrics

## Checking for Duplicate Rows

Duplicate rows in a clinical dataset can result from redundant data entry, exports from merged systems, or preprocessing artifacts. We check for exact duplicates across all columns and remove them if found.

Removing duplicates ensures:
- Each observation contributes equally to training
- Our model doesn’t overfit to repeated patterns

In [ ]:
# Count total duplicate rows
duplicate_count = df.duplicated().sum()
duplicate_count

In [ ]:
# Preview the first few duplicates
df[df.duplicated()].head()

In [ ]:
print(f"Original: {df.shape[0] + duplicate_count}, After deduplication: {df.shape[0]}")

In [ ]:
df = df.drop_duplicates()

We found and removed 0 duplicate rows.

Duplicates can occur in clinical studies due to repeated patient evaluations, export errors, or data merges. Their removal improves model generalization by ensuring each patient record is unique.

## Reviewing Numerical Feature Ranges

We check the distribution of all continuous variables to ensure:
- They lie within clinically plausible bounds
- There are no outliers due to entry errors or unit mismatches
- The model doesn’t learn from nonsensical or biologically impossible values

This step also helps us identify candidates for later transformation (e.g., log-scaling, normalization).

In [ ]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns

# Summarize them
df[numeric_cols].describe().T

In [ ]:
suspicious_rows = df[
    (df["vitamin_d_ng_ml"] < 4) | (df["vitamin_d_ng_ml"] > 100) |
    (df["glucose_mg_dl"] < 40) | (df["glucose_mg_dl"] > 400) |
    (df["bmi"] < 10) | (df["bmi"] > 70) |
    (df["lean_mass_percent"] < 20) | (df["lean_mass_percent"] > 95) |
    (df["fat_ratio_percent"] < 5) | (df["fat_ratio_percent"] > 80) |
    (df["visceral_fat_area_cm2"] < 0.5) | (df["visceral_fat_area_cm2"] > 45)
]

In [ ]:
print(f"Number of suspicious rows: {suspicious_rows.shape[0]}")
suspicious_rows.head()

In [ ]:
import numpy as np

flagged_columns = []

# Define bounds
bounds = {
    "vitamin_d_ng_ml": (4, 100),
    "glucose_mg_dl": (40, 400),
    "bmi": (10, 70),
    "lean_mass_percent": (20, 95),
    "fat_ratio_percent": (5, 80),
    "visceral_fat_area_cm2": (0.5, 45)
}

# Count violations per column
for col, (low, high) in bounds.items():
    count = ((df[col] < low) | (df[col] > high)).sum()
    flagged_columns.append((col, count))

flagged_columns = sorted(flagged_columns, key=lambda x: x[1], reverse=True)
for col, count in flagged_columns:
    print(f"{col}: {count} flagged rows")

In [ ]:
# Cap extreme glucose values at 400 mg/dL and flag them to reduce outlier influence while retaining the row.
df["glucose_outlier_flag"] = df["glucose_mg_dl"] > 400
df.loc[df["glucose_outlier_flag"], "glucose_mg_dl"] = 400

We examined continuous numeric features for biologically implausible values, using conservative yet clinically-informed thresholds. This process helps ensure our dataset is free from data entry errors or measurement artifacts that could distort model behavior.

### Threshold Summary

| Feature                   | Range Checked       | Justification |
|---------------------------|---------------------|----------------|
| `vitamin_d_ng_ml`         | <4 or >150          | <4 = severe deficiency, >150 = toxic/suspicious |
| `glucose_mg_dl`           | <40 or >400         | <40 = hypoglycemia, >400 = implausible hyperglycemia |
| `bmi`                     | <10 or >70          | Reflects extreme underweight or error |
| `lean_mass_percent`       | <20 or >95          | Normal physiological range |
| `fat_ratio_percent`       | <5 or >80           | Low and high extremes for body fat % |
| `visceral_fat_area_cm2`   | <0.5 or >45         | Adjusted based on observed distribution; likely device-scaled |

### Results

- **Two rows** were flagged across all checks:
  - One with `vitamin_d_ng_ml = 3.5`: retained as a valid case of extreme deficiency
  - One with `glucose_mg_dl = 575`: value was capped at 400 and flagged as an outlier

- `visceral_fat_area_cm2` initially triggered 120 flags using generic cm²-based thresholds. After reviewing the observed range (0.9–41), we revised bounds to <0.5 and >45. This resolved all false positives while preserving important physiologic variance.

### Conclusion

The dataset now contains only values within clinically realistic ranges. We retained edge-case records that reflect meaningful physiological extremes, and capped one implausible glucose value while preserving the associated row. No rows were dropped.

## Renaming Key Binary and Class Columns for Clarity

To enhance readability, consistency, and ease of reference throughout modeling and interpretation, we renamed several categorical and binary variables using short, lowercase, and descriptive identifiers.

This also aligns with clinical naming conventions and keeps features concise when used in plots, feature importances, and model documentation (e.g., model cards).

Below is a summary of renamed columns.

| Original Column                        | New Name                  | Reason for Rename |
|----------------------------------------|----------------------------|--------------------|
| `gallstone_status`                     | `has_gallstones`           | Target variable; renamed to reflect binary yes/no meaning clearly |
| `coronary_artery_disease_cad`          | `cad`                      | Shortens repetitive phrase, keeps common acronym |
| `diabetes_mellitus_dm`                 | `diabetes`                | Simplifies medical term, avoids acronym stacking |
| `hepatic_fat_accumulation_hfa`         | `hepatic_fat`             | Matches domain term: hepatic fat (liver fat) |
| `visceral_muscle_area_vma_kg`          | `visceral_muscle_mass_kg` | Clarifies value represents mass, not area |
| `visceral_fat_area_vfa`                | `visceral_fat_area_cm2`   | Clarifies unit and context |
| `total_fat_content_tfc`                | `fat_mass_kg`             | Defines quantity explicitly |
| `total_body_fat_ratio_tbfr`            | `fat_ratio_percent`       | Adds unit for clarity |
| `lean_mass_lm`                         | `lean_mass_percent`       | Adds unit and removes abbreviation |
| `body_protein_content_protein`         | `protein_percent`         | Adds unit, simplifies |
| `muscle_mass_mm`                       | `muscle_mass_kg`          | Defines unit and removes abbreviation |
| `bone_mass_bm`                         | `bone_mass_kg`            | Standardizes naming for consistency |

## Human-Centered Relevance Notes

This dataset was collected in a clinical setting using non-invasive tools such as bioimpedance analysis and routine laboratory blood tests. Our preprocessing and cleaning steps were informed by real-world usability and interpretability concerns.

### 1. Features are Non-Invasive and Accessible
- All inputs are derived from standard outpatient data (demographics, labs, body composition).
- No imaging, genomics, or protected identifiers were included.
- This ensures the model can be used in primary care or telehealth environments without requiring specialized equipment.

### 2. Features Align with Real Clinical Judgment
- Key predictors such as `vitamin_d_ng_ml`, `bmi`, `fat_ratio_percent`, and `hepatic_fat` reflect known risk factors for gallstone disease.
- Clinicians can interpret these features without needing ML expertise, which increases model explainability and trust.

### 3. Data Types Support Interpretability
- Categorical variables like `gender`, `comorbidity`, and `cad` were explicitly cast as category types for modeling clarity.
- Continuous variables are standardized and unit-annotated in column names (e.g., `_mg_dl`, `_kg`, `_percent`), making it easier to understand scale and risk implications.

### 4. No Target Leakage Detected
- We reviewed all features and confirmed that none reveal the outcome directly.
- The target variable `has_gallstones` is not derived from any included feature (e.g., no imaging reports or diagnostic codes were used as predictors).

### 5. Balanced Target Distribution
- The dataset is nearly evenly split between positive and negative cases.
- This supports fair and stable model training without synthetic sampling or reweighting.

### 6. Outliers Handled with Care
- Rare values were flagged, visually inspected, and resolved with domain-informed decisions (e.g., flagging and capping glucose at 400 mg/dL).
- This protects against biased model behavior without erasing physiologically meaningful cases.

### 7. Feature Names Are Human-Readable
- Long or machine-exported feature names were renamed for clarity (e.g., `diabetes_mellitus_dm` → `diabetes`, `gallstone_status` → `has_gallstones`).
- Units were added where appropriate to support safe interpretation (e.g., `glucose_mg_dl`).

Together, these actions support the development of a model that is not only accurate, but also responsible, understandable, and ready for real-world use.

## Saving Cleaned Dataset

After completing column standardization, data type correction, outlier handling, and human-centered review, we save the cleaned dataset to a structured location for use in downstream tasks such as exploratory data analysis (EDA), modeling, and evaluation.

This ensures:
- **Reproducibility**: All preprocessing is preserved in version-controlled form
- **Efficiency**: Clean data is loaded directly in later notebooks, avoiding repeated processing
- **Portability**: The file can be shared with collaborators or passed to pipelines without rerunning upstream cells

We save the dataset in CSV format using UTF-8 encoding, without an index column, and store it in the `data/` directory under the project root.

In [ ]:
df.to_csv("../data/cleaned.csv", index=False)

In [ ]:
# Confirm successful save
pd.read_csv("../data/cleaned.csv").shape

In [ ]:
df.dtypes.to_csv("../data/cleaned_dtypes.csv")
df.describe().T.to_csv("../data/cleaned_summary.csv")

The cleaned dataset is now saved as `cleaned.csv` and ready for use in:

- Day 3: Exploratory Data Analysis (EDA)
- Day 4: Baseline and advanced model training
- Day 6–7: Model interpretation and reporting

The dataset includes:
- 319 patients
- 40 features (32 numeric, 8 categorical)
- 50.5% gallstone-positive cases
- Key features include `bmi`, `vitamin_d_ng_ml`, `fat_ratio_percent`, and `hepatic_fat`

## PAIR: Mental Models – Key Takeaways

- Clinicians already have expectations (mental models) about how diagnostic tools behave—for example, they expect higher vitamin D or fat levels to correlate with certain conditions.
- When a model’s outputs and explanations align with these expectations, it feels more trustworthy and intuitive to use.
- In healthcare, mismatches between user intuition and system behavior can erode confidence or lead to dangerous misinterpretation.
- Therefore, we aim to design models where both **inputs** (e.g., interpretable lab values) and **outputs** (e.g., binary risk predictions) reinforce the clinician’s understanding and workflow—not challenge it.